In [1]:
import numpy as np
import pandas as pd
import cv2

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import sys
from tqdm import tqdm
from time import time
import shutil

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
import matplotlib.pyplot as plt

def plot_progress(history):
    loss_tr = []
    loss_vl = []
    acc_tr = []
    acc_vl = []
    loss_tr.append(history.history['loss'])
    loss_vl.append(history.history['val_loss'])
    acc_tr.append(history.history['categorical_accuracy'])
    acc_vl.append(history.history['val_categorical_accuracy'])
    
    plt.plot(acc_tr[0])
    plt.plot(acc_vl[0])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(loss_tr[0])
    plt.plot(loss_vl[0])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

In [3]:
from sklearn.metrics import classification_report,multilabel_confusion_matrix

In [6]:
inception1 = 0
def report(model):
    prediction = model.predict(test_gen)
    if model == inception1:
        print('incept')
        prediction = prediction[0]
    predicted_labels = np.argmax(prediction, axis=1)
    confusion_mat = multilabel_confusion_matrix(test_gen.labels, predicted_labels)
    print(confusion_mat)
    print(classification_report(test_gen.labels, predicted_labels))

In [17]:
train_path = '/kaggle/input/cwry-2/Model_Image/train'
val_path = '/kaggle/input/cwry-2/Model_Image/val'
test_path = '/kaggle/input/cwry-2/Model_Image/test'

num_train_samples = 12909
num_test_samples = 2815
num_val_samples = 2741
train_batch_size = 64
val_batch_size = 64


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [18]:
train_steps,val_steps

(202.0, 43.0)

In [19]:
IMAGE_SIZE = 256
def preprocess(img):
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    return img

datagen = ImageDataGenerator(preprocessing_function=lambda img: preprocess(img),
                             rescale=1.0/255)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(val_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

test_gen = datagen.flow_from_directory(test_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)

Found 12909 images belonging to 16 classes.
Found 2741 images belonging to 16 classes.
Found 2815 images belonging to 16 classes.


# DL Models

In [20]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, DepthwiseConv2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras import Model

In [21]:
# Metrics
cat_acc = tf.keras.metrics.CategoricalAccuracy()
auc_roc = tf.keras.metrics.AUC(multi_label=True, num_labels=16)
prec = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

In [22]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


## AlexNet

In [27]:
def AlexNet(input_shape):
    input = Input(shape=input_shape)
    
    x = Conv2D(96, (11, 11), strides=(4, 4), activation='relu')(input)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Conv2D(384, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.SpatialDropout2D(0.5)(x)
    x = BatchNormalization()(x)
    x = Conv2D(384, kernel_size=3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = BatchNormalization()(x)
    
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(16, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output, name='AlexNet')
    
    return model

In [31]:
alexnet = AlexNet(input_shape=(256,256,3))
alexnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [32]:
alexnet.count_params()

58352400

In [33]:
start = time()
alexnet_hist = alexnet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50


2023-08-27 19:02:19.307003: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inAlexNet/spatial_dropout2d_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


202/202 - 79s - loss: 2.4351 - categorical_accuracy: 0.2892 - auc: 0.7995 - precision: 0.4847 - recall: 0.1603 - val_loss: 6.0100 - val_categorical_accuracy: 0.0664 - val_auc: 0.5514 - val_precision: 0.0937 - val_recall: 0.0518 - 79s/epoch - 391ms/step
Epoch 2/50
202/202 - 74s - loss: 1.1290 - categorical_accuracy: 0.5877 - auc: 0.9460 - precision: 0.7210 - recall: 0.4773 - val_loss: 5.9648 - val_categorical_accuracy: 0.0660 - val_auc: 0.5815 - val_precision: 0.0851 - val_recall: 0.0657 - 74s/epoch - 365ms/step
Epoch 3/50
202/202 - 72s - loss: 0.5496 - categorical_accuracy: 0.8023 - auc: 0.9833 - precision: 0.8482 - recall: 0.7639 - val_loss: 2.3147 - val_categorical_accuracy: 0.5064 - val_auc: 0.8742 - val_precision: 0.5213 - val_recall: 0.4863 - 72s/epoch - 357ms/step
Epoch 4/50
202/202 - 73s - loss: 0.3134 - categorical_accuracy: 0.8883 - auc: 0.9927 - precision: 0.9075 - recall: 0.8743 - val_loss: 0.7127 - val_categorical_accuracy: 0.8201 - val_auc: 0.9762 - val_precision: 0.8267 -

In [34]:
hist_df = pd.DataFrame(alexnet_hist.history)
hist_json_file = 'AlexNet_50ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [154]:
alexnet.save_weights('AlexNet.hdf5')

In [35]:
alexnet.evaluate(test_gen)

2815/2815 [==============================] - 26s 9ms/step - loss: 0.8476 - categorical_accuracy: 0.9648 - auc: 0.9662 - precision: 0.9648 - recall: 0.9648


[0.8476091623306274,
 0.9648312330245972,
 0.9661738872528076,
 0.9648312330245972,
 0.9648312330245972]

In [36]:
report(alexnet)

2815/2815 [==============================] - 14s 5ms/step
[[[2623    6]
  [   2  184]]

 [[2629    0]
  [   2  184]]

 [[2629    0]
  [   9  177]]

 [[2608   21]
  [   0  186]]

 [[2578   51]
  [   0  186]]

 [[2621    8]
  [   1  185]]

 [[2626    3]
  [   1  185]]

 [[2629    0]
  [  10  176]]

 [[2623    6]
  [  14  172]]

 [[2629    0]
  [   0  186]]

 [[2625    4]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   1  185]]

 [[2754    0]
  [  59    2]]

 [[2753    0]
  [   0   62]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       186
           1       1.00      0.99      0.99       186
           2       1.00      0.95      0.98       186
           3       0.90      1.00      0.95       186
           4       0.78      1.00      0.88       186
           5       0.96      0.99      0.98       186
           6       0.98      0.99      0.99       186
           7       1.00      

## VGG

In [37]:
def conv_2(x,filters,kernel_size,activation):
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

def conv_3(x,filters,kernel_size,activation):
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

def conv_4(x,filters,kernel_size,activation):
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

def FC(x,units,activation):
    x = Flatten()(x) 
    x = Dense(units = units, activation =activation)(x)
    x = Dropout(0.5)(x)
    x = Dense(units = units, activation =activation)(x)
    x = Dropout(0.5)(x)
    x = Dense(units = 16, activation ='softmax')(x)
    return x

### VGG16

In [38]:
def VGG16(input_shape):
    
    input = Input(shape=input_shape)
    
    filters = 64
    kernel_same = 3
    act = 'relu'
    
    x = conv_2(input,filters,kernel_same,act)
    x = conv_2(x,filters*2,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_3(x,filters*4,kernel_same,act)
    x = conv_3(x,filters*8,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_3(x,filters*8,kernel_same,act)
    
    units = 4096
    
    output = FC(x,units,act)
    
    model = Model(inputs=input, outputs=output, name='VGG16')
    
    return model

In [39]:
vgg16 = VGG16(input_shape=(256,256,3))
vgg16.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [40]:
vgg16.count_params()

165783376

In [41]:
start = time()
vgg16_hist = vgg16.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50


2023-08-27 20:11:36.491384: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inVGG16/spatial_dropout2d_9/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


202/202 - 182s - loss: 2.4944 - categorical_accuracy: 0.3055 - auc: 0.7750 - precision: 0.9479 - recall: 0.2071 - val_loss: 1.7681 - val_categorical_accuracy: 0.3521 - val_auc: 0.8991 - val_precision: 1.0000 - val_recall: 0.1040 - 182s/epoch - 899ms/step
Epoch 2/50
202/202 - 150s - loss: 1.5180 - categorical_accuracy: 0.4431 - auc: 0.9058 - precision: 0.7561 - recall: 0.2406 - val_loss: 0.8948 - val_categorical_accuracy: 0.6753 - val_auc: 0.9652 - val_precision: 0.9129 - val_recall: 0.4743 - 150s/epoch - 745ms/step
Epoch 3/50
202/202 - 151s - loss: 0.8101 - categorical_accuracy: 0.6951 - auc: 0.9684 - precision: 0.8348 - recall: 0.5799 - val_loss: 0.3613 - val_categorical_accuracy: 0.8880 - val_auc: 0.9946 - val_precision: 0.9524 - val_recall: 0.8107 - 151s/epoch - 746ms/step
Epoch 4/50
202/202 - 152s - loss: 0.4152 - categorical_accuracy: 0.8420 - auc: 0.9899 - precision: 0.8907 - recall: 0.7998 - val_loss: 0.1931 - val_categorical_accuracy: 0.9362 - val_auc: 0.9979 - val_precision: 0

In [42]:
hist_df = pd.DataFrame(vgg16_hist.history)
hist_json_file = 'VGG16_50ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [43]:
vgg16.evaluate(test_gen)

2815/2815 [==============================] - 30s 11ms/step - loss: 58.1734 - categorical_accuracy: 0.5911 - auc: 0.7707 - precision: 0.5911 - recall: 0.5911


[58.173362731933594,
 0.5911189913749695,
 0.7706522941589355,
 0.5911189913749695,
 0.5911189913749695]

In [44]:
report(vgg16)

2815/2815 [==============================] - 20s 7ms/step
[[[2485  144]
  [  78  108]]

 [[2399  230]
  [   0  186]]

 [[2611   18]
  [ 134   52]]

 [[2537   92]
  [  62  124]]

 [[2629    0]
  [ 124   62]]

 [[2614   15]
  [  89   97]]

 [[2512  117]
  [  82  104]]

 [[2573   56]
  [  56  130]]

 [[2528  101]
  [  71  115]]

 [[2491  138]
  [   7  179]]

 [[2629    0]
  [ 162   24]]

 [[2391  238]
  [  31  155]]

 [[2627    2]
  [ 126   60]]

 [[2754    0]
  [  60    1]]

 [[2753    0]
  [  26   36]]

 [[2541    0]
  [  43  231]]]
              precision    recall  f1-score   support

           0       0.43      0.58      0.49       186
           1       0.45      1.00      0.62       186
           2       0.74      0.28      0.41       186
           3       0.57      0.67      0.62       186
           4       1.00      0.33      0.50       186
           5       0.87      0.52      0.65       186
           6       0.47      0.56      0.51       186
           7       0.70      

### VGG19

In [45]:
def VGG19(input_shape):
    
    input = Input(shape=input_shape)
    
    filters = 64
    kernel_same = 3
    act = 'relu'
    
    x = conv_2(input,filters,kernel_same,act)
    x = conv_2(x,filters*2,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_4(x,filters*4,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.5)(x)
    x = conv_4(x,filters*8,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_4(x,filters*8,kernel_same,act)
    
    units = 4096
    
    output = FC(x,units,act)
    
    model = Model(inputs=input, outputs=output)
    
    return model

In [46]:
vgg19 = VGG19(input_shape=(256,256,3))
vgg19.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [47]:
vgg19.count_params()

171093072

In [48]:
start = time()
vgg19_hist = vgg19.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50


2023-08-27 22:37:00.995097: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/spatial_dropout2d_11/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


202/202 - 233s - loss: 2.6299 - categorical_accuracy: 0.1974 - auc: 0.6690 - precision: 0.6220 - recall: 0.1216 - val_loss: 2.3057 - val_categorical_accuracy: 0.1722 - val_auc: 0.7678 - val_precision: 1.0000 - val_recall: 0.0992 - 233s/epoch - 1s/step
Epoch 2/50
202/202 - 179s - loss: 2.0285 - categorical_accuracy: 0.2723 - auc: 0.8128 - precision: 0.8474 - recall: 0.1132 - val_loss: 1.5368 - val_categorical_accuracy: 0.4068 - val_auc: 0.9106 - val_precision: 0.7658 - val_recall: 0.2160 - 179s/epoch - 887ms/step
Epoch 3/50
202/202 - 179s - loss: 1.3473 - categorical_accuracy: 0.4992 - auc: 0.9206 - precision: 0.7693 - recall: 0.3161 - val_loss: 0.9275 - val_categorical_accuracy: 0.6527 - val_auc: 0.9657 - val_precision: 0.8069 - val_recall: 0.4757 - 179s/epoch - 884ms/step
Epoch 4/50
202/202 - 186s - loss: 0.9395 - categorical_accuracy: 0.6506 - auc: 0.9601 - precision: 0.7946 - recall: 0.5207 - val_loss: 0.8951 - val_categorical_accuracy: 0.6538 - val_auc: 0.9715 - val_precision: 0.73

In [49]:
hist_df = pd.DataFrame(vgg19_hist.history)
hist_json_file = 'VGG19_50ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [50]:
pred = vgg19.evaluate(test_gen)

2815/2815 [==============================] - 34s 12ms/step - loss: 4682.5850 - categorical_accuracy: 0.1560 - auc: 0.5529 - precision: 0.1560 - recall: 0.1560


In [51]:
report(vgg19)

2815/2815 [==============================] - 24s 8ms/step
[[[1206 1423]
  [  24  162]]

 [[2529  100]
  [ 183    3]]

 [[2629    0]
  [ 186    0]]

 [[2592   37]
  [ 183    3]]

 [[2629    0]
  [ 186    0]]

 [[2629    0]
  [ 186    0]]

 [[2629    0]
  [ 186    0]]

 [[2225  404]
  [  25  161]]

 [[2312  317]
  [ 154   32]]

 [[2587   42]
  [ 184    2]]

 [[2629    0]
  [ 186    0]]

 [[2629    0]
  [ 186    0]]

 [[2576   53]
  [ 131   55]]

 [[2754    0]
  [  40   21]]

 [[2753    0]
  [  62    0]]

 [[2541    0]
  [ 274    0]]]
              precision    recall  f1-score   support

           0       0.10      0.87      0.18       186
           1       0.03      0.02      0.02       186
           2       0.00      0.00      0.00       186
           3       0.07      0.02      0.03       186
           4       0.00      0.00      0.00       186
           5       0.00      0.00      0.00       186
           6       0.00      0.00      0.00       186
           7       0.28      

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## MobileNet

In [70]:
def block(x, filters, strides, activation):
    
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = Conv2D(filters=filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    return x

def MobileNet(input_shape):
    
    activation = 'relu'
    filters = 32
    reps = 5
    input = Input(shape =input_shape)
    x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = block(x, filters = filters, strides = 1, activation=activation)
    x = block(x, filters = filters*2, strides = 2, activation=activation)
    x = block(x, filters = filters*2, strides = 1, activation=activation)
    x = block(x, filters = filters*4, strides = 2, activation=activation)
    x = Dropout(0.2)(x)
    x = block(x, filters = filters*4, strides = 1, activation=activation)
    x = block(x, filters = filters*8, strides = 2, activation=activation)
    for _ in range (reps):
         x = block(x, filters = filters*8, strides = 1, activation=activation)
    x = block(x, filters = filters*16, strides = 2, activation=activation)
    x = block(x, filters = filters*16, strides = 1, activation=activation)
    x = Dropout(0.2)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    output = Dense(units = 3, activation = 'softmax')(x)
    model = Model(inputs=input, outputs=output)
    
    return model

In [71]:
mobilenet = MobileNet(input_shape=(256,256,3))
mobilenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [72]:
mobilenet.count_params()

848379

In [73]:
start = time()
mobilenet_hist = mobilenet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

Epoch 1/10


2023-05-28 01:48:26.988870: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout_18/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


450/450 - 114s - loss: 0.2762 - categorical_accuracy: 0.9182 - auc: 0.9844 - auc_1: 0.9706 - val_loss: 3.2312 - val_categorical_accuracy: 0.3333 - val_auc: 0.5000 - val_auc_1: 0.3333 - 114s/epoch - 253ms/step

Epoch 2/10

450/450 - 95s - loss: 0.1696 - categorical_accuracy: 0.9319 - auc: 0.9898 - auc_1: 0.9792 - val_loss: 0.3577 - val_categorical_accuracy: 0.8647 - val_auc: 0.9872 - val_auc_1: 0.9762 - 95s/epoch - 211ms/step

Epoch 3/10

450/450 - 94s - loss: 0.1717 - categorical_accuracy: 0.9328 - auc: 0.9893 - auc_1: 0.9781 - val_loss: 0.2491 - val_categorical_accuracy: 0.8975 - val_auc: 0.9871 - val_auc_1: 0.9741 - 94s/epoch - 209ms/step

Epoch 4/10

450/450 - 93s - loss: 0.1573 - categorical_accuracy: 0.9351 - auc: 0.9912 - auc_1: 0.9827 - val_loss: 0.2802 - val_categorical_accuracy: 0.8970 - val_auc: 0.9874 - val_auc_1: 0.9756 - 93s/epoch - 206ms/step

Epoch 5/10

450/450 - 92s - loss: 0.1558 - categorical_accuracy: 0.9358 - auc: 0.9910 - auc_1: 0.9817 - val_loss: 0.2086 - val_cat

In [74]:
hist_df = pd.DataFrame(mobilenet_hist.history)
hist_json_file = 'MobileNet_10ep_histori.json'
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [75]:
pred = mobilenet.evaluate(test_gen)

3600/3600 [==============================] - 33s 9ms/step - loss: 0.1905 - categorical_accuracy: 0.9289 - auc: 0.9908 - auc_1: 0.9814


In [76]:
report(mobilenet)

3600/3600 [==============================] - 21s 6ms/step

[[[2182  218]

  [  36 1164]]



 [[2392    8]

  [ 135 1065]]



 [[2370   30]

  [  85 1115]]]

              precision    recall  f1-score   support



      faulty     0.8423    0.9700    0.9016      1200

      normal     0.9925    0.8875    0.9371      1200

     vfaulty     0.9738    0.9292    0.9510      1200



    accuracy                         0.9289      3600

   macro avg     0.9362    0.9289    0.9299      3600

weighted avg     0.9362    0.9289    0.9299      3600




## MobileNet v2

In [10]:
def inverted_residual_block(x, factor, filters_in, filters_out, strides, activation):
    
    x_add = x
    x = Conv2D(filters=filters_in*factor, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = Conv2D(filters=filters_out, kernel_size=1, strides=1,use_bias=False)(x)
    x = BatchNormalization()(x)
    
    if x_add.shape[-1] == x.shape[-1]:
        x = tf.keras.layers.Add()([x_add,x])
    
    return x

In [15]:
def MobileNet_v2(input_shape=(256,256,3)):
    
    input = Input(shape=input_shape)
    
    x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same', use_bias=False)(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu6')(x)
    
    x = inverted_residual_block(x, factor=1, filters_in=x.shape[-1], filters_out=16, strides=1, activation='relu6')
    x = Dropout(0.2)(x)
    for i in range(2):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=24, strides=s, activation='relu6')
    for i in range(3):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=32, strides=s, activation='relu6')
    for i in range(4):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=64, strides=s, activation='relu6')
    for i in range(3):
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=96, strides=1, activation='relu6')
    for i in range(3):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=160, strides=s, activation='relu6')
    x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=320, strides=1, activation='relu6')
    x = Dropout(0.2)(x)
    
    x = Conv2D(filters=1280, kernel_size=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu6')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    x = Flatten()(x)
    output = Dense(units=3, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output)
    
    return model

In [16]:
mobilenet_v2 = MobileNet_v2(input_shape=(256,256,3))
mobilenet_v2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [17]:
mobilenet_v2.count_params()

2277251

In [18]:
start = time()
mobilenet_v2_hist = mobilenet_v2.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

Epoch 1/10


2023-05-28 11:38:38.327724: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


450/450 - 155s - loss: 0.2516 - categorical_accuracy: 0.9112 - auc: 0.9832 - auc_1: 0.9671 - val_loss: 2.6948 - val_categorical_accuracy: 0.3333 - val_auc: 0.5000 - val_auc_1: 0.3333 - 155s/epoch - 344ms/step

Epoch 2/10

450/450 - 126s - loss: 0.1860 - categorical_accuracy: 0.9278 - auc: 0.9876 - auc_1: 0.9752 - val_loss: 0.2241 - val_categorical_accuracy: 0.9232 - val_auc: 0.9870 - val_auc_1: 0.9760 - 126s/epoch - 279ms/step

Epoch 3/10

450/450 - 123s - loss: 0.1742 - categorical_accuracy: 0.9322 - auc: 0.9889 - auc_1: 0.9776 - val_loss: 0.1920 - val_categorical_accuracy: 0.9310 - val_auc: 0.9878 - val_auc_1: 0.9769 - 123s/epoch - 273ms/step

Epoch 4/10

450/450 - 122s - loss: 0.1649 - categorical_accuracy: 0.9340 - auc: 0.9898 - auc_1: 0.9791 - val_loss: 0.2580 - val_categorical_accuracy: 0.9167 - val_auc: 0.9859 - val_auc_1: 0.9736 - 122s/epoch - 271ms/step

Epoch 5/10

450/450 - 122s - loss: 0.1571 - categorical_accuracy: 0.9384 - auc: 0.9908 - auc_1: 0.9807 - val_loss: 0.2181 - 

In [19]:
hist_df = pd.DataFrame(mobilenet_v2_hist.history)
hist_json_file = 'MobileNe_v2_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [20]:
pred = mobilenet_v2.evaluate(test_gen)

3600/3600 [==============================] - 48s 13ms/step - loss: 0.2512 - categorical_accuracy: 0.9333 - auc: 0.9858 - auc_1: 0.9724


In [23]:
report(mobilenet_v2)

3600/3600 [==============================] - 32s 9ms/step

[[[2199  201]

  [  36 1164]]



 [[2378   22]

  [  68 1132]]



 [[2383   17]

  [ 136 1064]]]

              precision    recall  f1-score   support



      faulty     0.8527    0.9700    0.9076      1200

      normal     0.9809    0.9433    0.9618      1200

     vfaulty     0.9843    0.8867    0.9329      1200



    accuracy                         0.9333      3600

   macro avg     0.9393    0.9333    0.9341      3600

weighted avg     0.9393    0.9333    0.9341      3600




## MobileNet v3

In [24]:
def bottleneck_block(x, kernel_size, stride, filters_out, exp_size, SE, NL):

    expanded_channels = exp_size
    reduction_ratio = 4
    x_add = x
    
    x = Conv2D(expanded_channels, kernel_size=1, padding='valid')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(NL)(x)
    
    x = DepthwiseConv2D(kernel_size=kernel_size, padding='same', strides=stride)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(NL)(x)
    
    if SE == True:
        # Squeeze and Excitation Layer
        squeeze = tf.keras.layers.GlobalAveragePooling2D()(x)
        excitation = Dense(expanded_channels // reduction_ratio, activation='relu')(squeeze)
        excitation = Dense(expanded_channels, activation='hard_sigmoid')(excitation)
        excitation = tf.keras.layers.Reshape((1, 1, expanded_channels))(excitation)
        x = tf.keras.layers.multiply([x, excitation])

    x = Conv2D(filters_out, kernel_size=1, padding='valid')(x)
    x = BatchNormalization()(x)
    
    if x_add.shape == x.shape:
        x = tf.keras.layers.add([x, x_add])
    
    return x

def efficient_final_layers(x,l=960):
    x = Conv2D(l, kernel_size=1, padding='valid', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('swish')(x)
    
    x = tf.keras.layers.AveragePooling2D(8)(x)
    
    x = Conv2D(1280, kernel_size=1, use_bias=False)(x)
    x = tf.keras.layers.Activation('swish')(x)
    x = Dropout(0.2)(x)
    x = Conv2D(3, kernel_size=1, activation='softmax')(x)
    
    x = tf.squeeze(x, 1)
    x = tf.squeeze(x, 1)
    
    return x

In [25]:
def MobileNet_v3(size='Large',input_shape=(256,256,3)):
    
    input = Input(shape=input_shape)
    
    if size == 'Large':
        x = Conv2D(16, kernel_size=3, padding='same', strides=2, use_bias=False)(input)
        x = BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)

                           # x  k  s  out exp sq&ex   act
        x = bottleneck_block(x, 3, 1, 16, 16, False, 'relu')
        x = bottleneck_block(x, 3, 2, 24, 64, False, 'relu')
        x = bottleneck_block(x, 3, 1, 24, 72, False, 'relu')
        x = Dropout(0.2)(x)

        x = bottleneck_block(x, 5, 2, 40, 72, True, 'relu')
        x = bottleneck_block(x, 5, 1, 40, 120, True, 'relu')
        x = bottleneck_block(x, 5, 1, 40, 120, True, 'relu')

        x = bottleneck_block(x, 3, 2, 80, 240, False, 'swish')
        x = bottleneck_block(x, 3, 1, 80, 200, False, 'swish')
        x = bottleneck_block(x, 3, 1, 80, 184, False, 'swish')
        x = bottleneck_block(x, 3, 1, 80, 184, False, 'swish')
        x = bottleneck_block(x, 3, 1, 112, 480, True, 'swish')
        x = bottleneck_block(x, 3, 1, 112, 672, True, 'swish')
        x = Dropout(0.2)(x)

        x = bottleneck_block(x, 5, 2, 160, 672, True, 'swish')
        x = bottleneck_block(x, 5, 1, 160, 960, True, 'swish')
        x = bottleneck_block(x, 5, 1, 160, 960, True, 'swish')
        

        output = efficient_final_layers(x)

        model = Model(inputs=input, outputs=output, name='MobileNetv3_large')
    if size == 'Small':
        x = Conv2D(16, kernel_size=3, padding='same', strides=2, use_bias=False)(input)
        x = BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)

                           # x  k  s  out exp sq&ex   act
        x = bottleneck_block(x, 3, 2, 16, 16, True, 'relu')
        x = bottleneck_block(x, 3, 2, 24, 72, False, 'relu')
        x = bottleneck_block(x, 3, 1, 24, 88, False, 'relu')
        x = Dropout(0.2)(x)
        
        x = bottleneck_block(x, 5, 2, 40, 96, True, 'swish')
        x = bottleneck_block(x, 5, 1, 40, 240, True, 'swish')
        x = bottleneck_block(x, 5, 1, 40, 240, True, 'swish')
        x = bottleneck_block(x, 5, 1, 48, 120, True, 'swish')
        x = bottleneck_block(x, 5, 1, 48, 144, True, 'swish')
        x = bottleneck_block(x, 5, 2, 96, 288, True, 'swish')
        x = Dropout(0.2)(x)
        x = bottleneck_block(x, 5, 1, 96, 576, True, 'swish')
        x = bottleneck_block(x, 5, 1, 96, 576, True, 'swish')

        output = efficient_final_layers(x,l=576)

        model = Model(inputs=input, outputs=output, name='MobileNetv3_Small')
    
    return model

In [26]:
Lmobilenet_v3 = MobileNet_v3(size='Large',input_shape=(256,256,3))
Lmobilenet_v3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [27]:
Lmobilenet_v3.count_params()

4238721

In [28]:
start = time()
Lmobilenet_v3_hist = Lmobilenet_v3.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

Epoch 1/10


2023-05-28 12:05:28.196987: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inMobileNetv3_large/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


450/450 - 155s - loss: 0.2099 - categorical_accuracy: 0.9226 - auc: 0.9843 - auc_1: 0.9684 - val_loss: 2.3968 - val_categorical_accuracy: 0.3333 - val_auc: 0.5000 - val_auc_1: 0.3333 - 155s/epoch - 345ms/step

Epoch 2/10

450/450 - 110s - loss: 0.1610 - categorical_accuracy: 0.9347 - auc: 0.9906 - auc_1: 0.9814 - val_loss: 0.1731 - val_categorical_accuracy: 0.9250 - val_auc: 0.9905 - val_auc_1: 0.9808 - 110s/epoch - 246ms/step

Epoch 3/10

450/450 - 109s - loss: 0.1495 - categorical_accuracy: 0.9389 - auc: 0.9916 - auc_1: 0.9826 - val_loss: 0.1660 - val_categorical_accuracy: 0.9382 - val_auc: 0.9906 - val_auc_1: 0.9822 - 109s/epoch - 243ms/step

Epoch 4/10

450/450 - 109s - loss: 0.1366 - categorical_accuracy: 0.9461 - auc: 0.9928 - auc_1: 0.9853 - val_loss: 0.1749 - val_categorical_accuracy: 0.9473 - val_auc: 0.9894 - val_auc_1: 0.9796 - 109s/epoch - 242ms/step

Epoch 5/10

450/450 - 109s - loss: 0.1347 - categorical_accuracy: 0.9466 - auc: 0.9927 - auc_1: 0.9849 - val_loss: 0.1492 - 

In [29]:
hist_df = pd.DataFrame(Lmobilenet_v3_hist.history)
hist_json_file = 'LMobileNe_v3_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [30]:
pred = Lmobilenet_v3.evaluate(test_gen)

3600/3600 [==============================] - 48s 13ms/step - loss: 0.1384 - categorical_accuracy: 0.9472 - auc: 0.9935 - auc_1: 0.9874


In [31]:
report(Lmobilenet_v3)

3600/3600 [==============================] - 39s 10ms/step

[[[2290  110]

  [  78 1122]]



 [[2342   58]

  [  25 1175]]



 [[2378   22]

  [  87 1113]]]

              precision    recall  f1-score   support



      faulty     0.9107    0.9350    0.9227      1200

      normal     0.9530    0.9792    0.9659      1200

     vfaulty     0.9806    0.9275    0.9533      1200



    accuracy                         0.9472      3600

   macro avg     0.9481    0.9472    0.9473      3600

weighted avg     0.9481    0.9472    0.9473      3600




In [32]:
Smobilenet_v3 = MobileNet_v3(size='Small',input_shape=(256,256,3))
Smobilenet_v3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [33]:
Smobilenet_v3.count_params()

1680425

In [35]:
start = time()
Smobilenet_v3_hist = Smobilenet_v3.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

Epoch 1/10


2023-05-28 12:32:23.947130: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inMobileNetv3_Small/dropout_7/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


450/450 - 132s - loss: 0.2223 - categorical_accuracy: 0.9199 - auc: 0.9862 - auc_1: 0.9732 - val_loss: 1.9227 - val_categorical_accuracy: 0.3333 - val_auc: 0.6413 - val_auc_1: 0.5109 - 132s/epoch - 293ms/step

Epoch 2/10

450/450 - 86s - loss: 0.1543 - categorical_accuracy: 0.9401 - auc: 0.9910 - auc_1: 0.9815 - val_loss: 0.1351 - val_categorical_accuracy: 0.9490 - val_auc: 0.9927 - val_auc_1: 0.9850 - 86s/epoch - 191ms/step

Epoch 3/10

450/450 - 85s - loss: 0.1362 - categorical_accuracy: 0.9460 - auc: 0.9930 - auc_1: 0.9860 - val_loss: 0.1269 - val_categorical_accuracy: 0.9518 - val_auc: 0.9937 - val_auc_1: 0.9876 - 85s/epoch - 189ms/step

Epoch 4/10

450/450 - 83s - loss: 0.1191 - categorical_accuracy: 0.9536 - auc: 0.9944 - auc_1: 0.9888 - val_loss: 0.1304 - val_categorical_accuracy: 0.9515 - val_auc: 0.9939 - val_auc_1: 0.9880 - 83s/epoch - 184ms/step

Epoch 5/10

450/450 - 80s - loss: 0.1097 - categorical_accuracy: 0.9564 - auc: 0.9951 - auc_1: 0.9903 - val_loss: 0.1486 - val_cat

In [36]:
hist_df = pd.DataFrame(Smobilenet_v3_hist.history)
hist_json_file = 'SMobileNe_v3_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [37]:
pred = Smobilenet_v3.evaluate(test_gen)

3600/3600 [==============================] - 45s 12ms/step - loss: 0.1418 - categorical_accuracy: 0.9514 - auc: 0.9925 - auc_1: 0.9850


In [38]:
report(Smobilenet_v3)

3600/3600 [==============================] - 34s 9ms/step

[[[2330   70]

  [ 103 1097]]



 [[2379   21]

  [  43 1157]]



 [[2316   84]

  [  29 1171]]]

              precision    recall  f1-score   support



      faulty     0.9400    0.9142    0.9269      1200

      normal     0.9822    0.9642    0.9731      1200

     vfaulty     0.9331    0.9758    0.9540      1200



    accuracy                         0.9514      3600

   macro avg     0.9518    0.9514    0.9513      3600

weighted avg     0.9518    0.9514    0.9513      3600




## ResNet under 50

In [39]:
def identity_block(x, filter):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    
    return x

def convolutional_block(x, filter):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same', strides=2)(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    
    x_skip = Conv2D(filters=filter, kernel_size=1, strides=2)(x_skip)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    return x

In [40]:
def ResNet(layers=18, input_shape=(256,256,3)):
    
    filters = 64
    act = 'relu'
    
    input = Input(shape=input_shape)
    #x = tf.keras.layers.ZeroPadding2D((3,3))(x_input)
    
    x = Conv2D(filters=filters, kernel_size=(7,7), strides=2)(input)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(act)(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    if layers == 18:
        block_layers = [2,2,2,2]
    if layers == 34:
        block_layers = [3,4,6,3]
    for i in range(4):
        if i == 0:
            for j in range(block_layers[i]):
                x = identity_block(x, filters)
        else:
            filters = filters*2
            x = convolutional_block(x, filters)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filters)
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    output = Dense(units=3, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output, name='ResNet18')
    
    return model

In [41]:
resnet18 = ResNet(input_shape=(256,256,3))
resnet18.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [42]:
resnet18.count_params()

11188867

In [43]:
start = time()
resnet18_hist = resnet18.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

Epoch 1/10


2023-05-28 12:52:09.884609: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inResNet18/dropout_10/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


450/450 - 113s - loss: 0.2254 - categorical_accuracy: 0.9264 - auc: 0.9851 - auc_1: 0.9710 - val_loss: 1.1994 - val_categorical_accuracy: 0.3688 - val_auc: 0.9107 - val_auc_1: 0.8639 - 113s/epoch - 252ms/step

Epoch 2/10

450/450 - 80s - loss: 0.1649 - categorical_accuracy: 0.9332 - auc: 0.9902 - auc_1: 0.9804 - val_loss: 0.1659 - val_categorical_accuracy: 0.9353 - val_auc: 0.9915 - val_auc_1: 0.9832 - 80s/epoch - 178ms/step

Epoch 3/10

450/450 - 86s - loss: 0.1481 - categorical_accuracy: 0.9426 - auc: 0.9918 - auc_1: 0.9836 - val_loss: 0.2725 - val_categorical_accuracy: 0.9225 - val_auc: 0.9830 - val_auc_1: 0.9706 - 86s/epoch - 190ms/step

Epoch 4/10

450/450 - 81s - loss: 0.1302 - categorical_accuracy: 0.9483 - auc: 0.9932 - auc_1: 0.9860 - val_loss: 0.1542 - val_categorical_accuracy: 0.9372 - val_auc: 0.9925 - val_auc_1: 0.9849 - 81s/epoch - 180ms/step

Epoch 5/10

450/450 - 82s - loss: 0.1179 - categorical_accuracy: 0.9535 - auc: 0.9944 - auc_1: 0.9890 - val_loss: 0.1728 - val_cat

In [44]:
hist_df = pd.DataFrame(resnet18_hist.history)
hist_json_file = 'ResNet18_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [45]:
pred = resnet18.evaluate(test_gen)

3600/3600 [==============================] - 35s 10ms/step - loss: 0.1444 - categorical_accuracy: 0.9500 - auc: 0.9939 - auc_1: 0.9878


In [46]:
report(resnet18)

3600/3600 [==============================] - 24s 7ms/step

[[[2294  106]

  [  72 1128]]



 [[2389   11]

  [  55 1145]]



 [[2337   63]

  [  53 1147]]]

              precision    recall  f1-score   support



      faulty     0.9141    0.9400    0.9269      1200

      normal     0.9905    0.9542    0.9720      1200

     vfaulty     0.9479    0.9558    0.9519      1200



    accuracy                         0.9500      3600

   macro avg     0.9508    0.9500    0.9502      3600

weighted avg     0.9508    0.9500    0.9502      3600




In [47]:
resnet34 = ResNet(layers=34,input_shape=(256,256,3))
resnet34.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [48]:
resnet34.count_params()

21308163

In [49]:
start = time()
resnet34_hist = resnet34.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

Epoch 1/10


2023-05-28 13:11:59.818062: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inResNet18/dropout_12/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


450/450 - 113s - loss: 0.2523 - categorical_accuracy: 0.9206 - auc: 0.9825 - auc_1: 0.9654 - val_loss: 1.9612 - val_categorical_accuracy: 0.3333 - val_auc: 0.8196 - val_auc_1: 0.7452 - 113s/epoch - 252ms/step

Epoch 2/10

450/450 - 94s - loss: 0.1603 - categorical_accuracy: 0.9353 - auc: 0.9905 - auc_1: 0.9810 - val_loss: 0.3012 - val_categorical_accuracy: 0.8830 - val_auc: 0.9883 - val_auc_1: 0.9765 - 94s/epoch - 208ms/step

Epoch 3/10

450/450 - 90s - loss: 0.1411 - categorical_accuracy: 0.9446 - auc: 0.9923 - auc_1: 0.9846 - val_loss: 0.2795 - val_categorical_accuracy: 0.9305 - val_auc: 0.9847 - val_auc_1: 0.9714 - 90s/epoch - 201ms/step

Epoch 4/10

450/450 - 91s - loss: 0.1349 - categorical_accuracy: 0.9489 - auc: 0.9927 - auc_1: 0.9851 - val_loss: 0.1528 - val_categorical_accuracy: 0.9417 - val_auc: 0.9923 - val_auc_1: 0.9837 - 91s/epoch - 203ms/step

Epoch 5/10

450/450 - 94s - loss: 0.1263 - categorical_accuracy: 0.9519 - auc: 0.9936 - auc_1: 0.9870 - val_loss: 0.1960 - val_cat

In [50]:
hist_df = pd.DataFrame(resnet34_hist.history)
hist_json_file = 'ResNet34_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [51]:
pred = resnet34.evaluate(test_gen)

3600/3600 [==============================] - 44s 12ms/step - loss: 0.1672 - categorical_accuracy: 0.9364 - auc: 0.9902 - auc_1: 0.9814


In [52]:
report(resnet34)

3600/3600 [==============================] - 31s 8ms/step

[[[2268  132]

  [  95 1105]]



 [[2373   27]

  [  65 1135]]



 [[2330   70]

  [  69 1131]]]

              precision    recall  f1-score   support



      faulty     0.8933    0.9208    0.9069      1200

      normal     0.9768    0.9458    0.9610      1200

     vfaulty     0.9417    0.9425    0.9421      1200



    accuracy                         0.9364      3600

   macro avg     0.9373    0.9364    0.9367      3600

weighted avg     0.9373    0.9364    0.9367      3600




## ResNet above 50

In [12]:
def BN_identity_block(x, filter):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(1,1), padding='valid')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter*4, kernel_size=(1,1), padding='valid')(x)
    x = BatchNormalization(axis=3)(x)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    
    return x

def BN_convolutional_block(x, filter, s):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(1,1), padding='valid', strides=s)(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter*4, kernel_size=(1,1), padding='valid')(x)
    x = BatchNormalization(axis=3)(x)
    
    x_skip = Conv2D(filters=filter*4, kernel_size=1, strides=s, padding='valid')(x_skip)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    return x

In [13]:
def BN_ResNet(input_shape=(256,256,3)):
    
    filters = 64
    act = 'relu'
    
    x_input = Input(shape=input_shape)
    
    x = Conv2D(filters=filters, kernel_size=(7,7), strides=2)(x_input)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(act)(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    block_layers = [3,4,6,3]
    for i in range(4):
        if i == 0:
            x = BN_convolutional_block(x=x, filter=filters, s=1)
            for j in range(block_layers[i]-1):
                x = BN_identity_block(x, filters)
        else:
            filters = filters*2
            x = BN_convolutional_block(x, filters, s=2)
            for j in range(block_layers[i] - 1):
                x = BN_identity_block(x, filters)
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    output = Dense(units=3, activation='softmax')(x)
    
    model = Model(inputs=x_input, outputs=output, name='ResNet50')
    
    return model

In [14]:
resnet50 = BN_ResNet(input_shape=(256,256,3))
resnet50.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [15]:
resnet50.count_params()

23578499

In [16]:
start = time()
resnet50_hist = resnet50.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

Epoch 1/10


2023-05-28 15:56:31.344234: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inResNet50/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


450/450 - 246s - loss: 0.2684 - categorical_accuracy: 0.9090 - auc: 0.9789 - auc_1: 0.9589 - val_loss: 4.0934 - val_categorical_accuracy: 0.3333 - val_auc: 0.5522 - val_auc_1: 0.3739 - 246s/epoch - 546ms/step

Epoch 2/10

450/450 - 136s - loss: 0.1894 - categorical_accuracy: 0.9262 - auc: 0.9874 - auc_1: 0.9754 - val_loss: 0.3687 - val_categorical_accuracy: 0.8697 - val_auc: 0.9873 - val_auc_1: 0.9740 - 136s/epoch - 302ms/step

Epoch 3/10

450/450 - 136s - loss: 0.1672 - categorical_accuracy: 0.9298 - auc: 0.9901 - auc_1: 0.9805 - val_loss: 0.2664 - val_categorical_accuracy: 0.9070 - val_auc: 0.9862 - val_auc_1: 0.9725 - 136s/epoch - 303ms/step

Epoch 4/10

450/450 - 136s - loss: 0.1618 - categorical_accuracy: 0.9371 - auc: 0.9903 - auc_1: 0.9802 - val_loss: 0.2849 - val_categorical_accuracy: 0.8988 - val_auc: 0.9878 - val_auc_1: 0.9761 - 136s/epoch - 303ms/step

Epoch 5/10

450/450 - 138s - loss: 0.1450 - categorical_accuracy: 0.9438 - auc: 0.9922 - auc_1: 0.9844 - val_loss: 0.3305 - 

In [17]:
hist_df = pd.DataFrame(resnet50_hist.history)
hist_json_file = 'ResNet50_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [18]:
pred = resnet50.evaluate(test_gen)

3600/3600 [==============================] - 51s 14ms/step - loss: 0.2040 - categorical_accuracy: 0.9258 - auc: 0.9935 - auc_1: 0.9864


In [19]:
report(resnet50)

3600/3600 [==============================] - 37s 10ms/step

[[[2366   34]

  [ 228  972]]



 [[2370   30]

  [  33 1167]]



 [[2197  203]

  [   6 1194]]]

              precision    recall  f1-score   support



      faulty     0.9662    0.8100    0.8812      1200

      normal     0.9749    0.9725    0.9737      1200

     vfaulty     0.8547    0.9950    0.9195      1200



    accuracy                         0.9258      3600

   macro avg     0.9319    0.9258    0.9248      3600

weighted avg     0.9319    0.9258    0.9248      3600




## DenseNet

In [14]:
def convolutional_layer(x, filters, kernel=1, strides=1):
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = Conv2D(filters, kernel_size=kernel, strides=strides, padding='same')(x)
    return x

def dense_block(x, layers, filters):
    for _ in range(layers):
        x_d = convolutional_layer(x, 4*filters)
        x_d = convolutional_layer(x_d, filters, kernel=3)
        x = tf.keras.layers.Concatenate()([x_d,x])
    return x

def transition_layer(x):
    x = convolutional_layer(x, filters=x.shape[-1]//2)
    x = tf.keras.layers.AveragePooling2D(2, strides=2, padding='same')(x)
    return x

In [18]:
def denseNet121(input_shape):
    input = Input(shape=input_shape)
    
    x = Conv2D(filters=64, kernel_size=7, strides=2, padding='same')(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = MaxPool2D(3, strides=2, padding='same')(x)
    
    for l in [6,12,24,16]:
        d = dense_block(x, layers=l, filters=32)
        x = Dropout(0.2)(x)
        x = transition_layer(d)
    x = Dropout(0.5)(x)
        
    x = tf.keras.layers.GlobalAveragePooling2D()(d)
    x = Flatten()(x)
    output = Dense(units=3, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output, name='DenseNet')
    
    return model

In [19]:
densenet = denseNet121(input_shape=(256,256,3))
densenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [20]:
densenet.count_params()

7046723

In [21]:
start = time()
densenet_hist = densenet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

Epoch 1/10

450/450 - 313s - loss: 0.1928 - categorical_accuracy: 0.9266 - auc: 0.9867 - auc_1: 0.9735 - val_loss: 1.6162 - val_categorical_accuracy: 0.3333 - val_auc: 0.8541 - val_auc_1: 0.7849 - 313s/epoch - 696ms/step

Epoch 2/10

450/450 - 142s - loss: 0.1505 - categorical_accuracy: 0.9410 - auc: 0.9914 - auc_1: 0.9824 - val_loss: 0.1634 - val_categorical_accuracy: 0.9448 - val_auc: 0.9919 - val_auc_1: 0.9845 - 142s/epoch - 316ms/step

Epoch 3/10

450/450 - 143s - loss: 0.1363 - categorical_accuracy: 0.9479 - auc: 0.9927 - auc_1: 0.9850 - val_loss: 0.1660 - val_categorical_accuracy: 0.9428 - val_auc: 0.9903 - val_auc_1: 0.9812 - 143s/epoch - 317ms/step

Epoch 4/10

450/450 - 143s - loss: 0.1230 - categorical_accuracy: 0.9528 - auc: 0.9940 - auc_1: 0.9880 - val_loss: 0.3750 - val_categorical_accuracy: 0.8895 - val_auc: 0.9797 - val_auc_1: 0.9620 - 143s/epoch - 319ms/step

Epoch 5/10

450/450 - 143s - loss: 0.1208 - categorical_accuracy: 0.9543 - auc: 0.9941 - auc_1: 0.9881 - val_los

In [23]:
hist_df = pd.DataFrame(densenet_hist.history)
hist_json_file = 'densenet_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [24]:
pred = densenet.evaluate(test_gen)

3600/3600 [==============================] - 86s 24ms/step - loss: 0.1067 - categorical_accuracy: 0.9594 - auc: 0.9958 - auc_1: 0.9919


In [25]:
report(densenet)

3600/3600 [==============================] - 76s 20ms/step

[[[2326   74]

  [  69 1131]]



 [[2372   28]

  [  24 1176]]



 [[2356   44]

  [  53 1147]]]

              precision    recall  f1-score   support



      faulty     0.9386    0.9425    0.9405      1200

      normal     0.9767    0.9800    0.9784      1200

     vfaulty     0.9631    0.9558    0.9594      1200



    accuracy                         0.9594      3600

   macro avg     0.9595    0.9594    0.9594      3600

weighted avg     0.9595    0.9594    0.9594      3600




## Inception v1

In [21]:
def inception_module(x,filters_1x1,filters_3x3_reduce,filters_3x3,
    filters_5x5_reduce,filters_5x5,filters_pool_proj,name=None):

    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)

    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)
    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)
    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)
    output = tf.keras.layers.Concatenate(axis=3, name=name)([conv_1x1, conv_3x3, conv_5x5, pool_proj])

    return output

In [22]:
def inception_v1(input_shape=(256,256,3)):
    
    input = Input(shape=input_shape)
    
    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(input)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
    x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
    x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)
    
    x = inception_module(x,filters_1x1=64,filters_3x3_reduce=96,
     filters_3x3=128,filters_5x5_reduce=16,filters_5x5=32,filters_pool_proj=32,name='inception_3a')
    x = Dropout(0.2)(x)
    
    x = inception_module(x,filters_1x1=128,filters_3x3_reduce=128,
     filters_3x3=192,filters_5x5_reduce=32,filters_5x5=96,filters_pool_proj=64,name='inception_3b')
    x = Dropout(0.2)(x)
    
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)
    x = inception_module(x,filters_1x1=192,filters_3x3_reduce=96,
    filters_3x3=208,filters_5x5_reduce=16,filters_5x5=48,filters_pool_proj=64, name='inception_4a')
    x = Dropout(0.2)(x)
    
    x1 = tf.keras.layers.AveragePooling2D((5, 5), strides=3)(x)
    x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
    x1 = Flatten()(x1)
    x1 = Dense(1024, activation='relu')(x1)
    x1 = Dropout(0.7)(x1)
    x1 = Dense(3, activation='softmax', name='auxilliary_output_1')(x1)
    
    x = inception_module(x,filters_1x1=160,filters_3x3_reduce=112,
    filters_3x3=224,filters_5x5_reduce=24,filters_5x5=64,filters_pool_proj=64,name='inception_4b')
    x = Dropout(0.2)(x)
    
    x = inception_module(x,filters_1x1=128,filters_3x3_reduce=128,
     filters_3x3=256,filters_5x5_reduce=24,filters_5x5=64,filters_pool_proj=64,name='inception_4c')
    x = Dropout(0.2)(x)
    
    x = inception_module(x,filters_1x1=112,filters_3x3_reduce=144,
     filters_3x3=288,filters_5x5_reduce=32,filters_5x5=64,filters_pool_proj=64,name='inception_4d')
    x = Dropout(0.2)(x)
    
    x2 = tf.keras.layers.AveragePooling2D((5, 5), strides=3)(x)
    x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1024, activation='relu')(x2)
    x2 = Dropout(0.7)(x2)
    x2 = Dense(3, activation='softmax', name='auxilliary_output_2')(x2)
    
    x = inception_module(x,filters_1x1=256,filters_3x3_reduce=160,
     filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,filters_pool_proj=128,name='inception_4e')
    
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)
    x = inception_module(x,filters_1x1=256,filters_3x3_reduce=160,
     filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,filters_pool_proj=128,name='inception_5a')
    
    x = inception_module(x, filters_1x1=384,filters_3x3_reduce=192,
     filters_3x3=384,filters_5x5_reduce=48,filters_5x5=128,filters_pool_proj=128,name='inception_5b')
    
    x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)
    x = Dropout(0.4)(x)
    x = Dense(3, activation='softmax', name='output')(x)
    
    model = Model(inputs=input, outputs=[x, x1, x2], name='inception_v1')
    
    return model

In [23]:
inception1 = inception_v1(input_shape=(256,256,3))
inception1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=['categorical_crossentropy','categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1.0,0.3,0.3], metrics=[cat_acc,auc_roc,prec,recall])

In [24]:
start = time()
inception1_hist = inception1.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

Epoch 1/10


2023-05-27 23:32:49.942303: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ininception_v1/dropout_5/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


450/450 - 117s - loss: 0.5635 - output_loss: 0.3411 - auxilliary_output_1_loss: 0.3954 - auxilliary_output_2_loss: 0.3456 - output_categorical_accuracy: 0.8497 - output_auc: 0.9636 - output_auc_1: 0.9325 - auxilliary_output_1_categorical_accuracy: 0.8415 - auxilliary_output_1_auc: 0.9573 - auxilliary_output_1_auc_1: 0.9232 - auxilliary_output_2_categorical_accuracy: 0.8572 - auxilliary_output_2_auc: 0.9631 - auxilliary_output_2_auc_1: 0.9316 - val_loss: 0.6677 - val_output_loss: 0.4629 - val_auxilliary_output_1_loss: 0.3059 - val_auxilliary_output_2_loss: 0.3768 - val_output_categorical_accuracy: 0.8843 - val_output_auc: 0.9661 - val_output_auc_1: 0.9318 - val_auxilliary_output_1_categorical_accuracy: 0.8870 - val_auxilliary_output_1_auc: 0.9824 - val_auxilliary_output_1_auc_1: 0.9495 - val_auxilliary_output_2_categorical_accuracy: 0.8915 - val_auxilliary_output_2_auc: 0.9701 - val_auxilliary_output_2_auc_1: 0.9311 - 117s/epoch - 259ms/step

Epoch 2/10

450/450 - 83s - loss: 0.3314 - o

In [26]:
hist_df = pd.DataFrame(inception1_hist.history)
hist_json_file = 'inception1_10ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [27]:
pred = inception1.evaluate(test_gen)

3600/3600 [==============================] - 64s 18ms/step - loss: 0.3563 - output_loss: 0.2588 - auxilliary_output_1_loss: 0.1351 - auxilliary_output_2_loss: 0.1901 - output_categorical_accuracy: 0.9392 - output_auc: 0.9936 - output_auc_1: 0.9872 - auxilliary_output_1_categorical_accuracy: 0.9508 - auxilliary_output_1_auc: 0.9943 - auxilliary_output_1_auc_1: 0.9887 - auxilliary_output_2_categorical_accuracy: 0.9439 - auxilliary_output_2_auc: 0.9939 - auxilliary_output_2_auc_1: 0.9878


In [65]:
report(inception1)

3600/3600 [==============================] - 35s 10ms/step

incept

[[[2302   98]

  [ 117 1083]]



 [[2394    6]

  [  81 1119]]



 [[2285  115]

  [  21 1179]]]

              precision    recall  f1-score   support



      faulty     0.9170    0.9025    0.9097      1200

      normal     0.9947    0.9325    0.9626      1200

     vfaulty     0.9111    0.9825    0.9455      1200



    accuracy                         0.9392      3600

   macro avg     0.9409    0.9392    0.9393      3600

weighted avg     0.9409    0.9392    0.9393      3600


